<a href="https://colab.research.google.com/github/zoeebe05/3hentai-downloader-to-drive-colab/blob/main/NineMangaDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install colorama

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

import requests
import concurrent.futures
import os
import math
import re
import sys
import time
import zipfile
import shutil
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
from pathlib import Path
from typing import Optional
from colorama import init, Fore, Back, Style

from colorama import Fore, Style

class TuClase:
    def search_manga(self):
        language = "es"
        while True:
            print(Fore.BLUE + Style.BRIGHT + "--- MENU ---" + Fore.RESET)

            menu_options = [
                [Fore.BLUE + Fore.YELLOW + "1" + Fore.RESET, f"Select Language ({language})"],
                [Fore.BLUE + Fore.YELLOW + "2" + Fore.RESET, "Search Manga"],
                [Fore.BLUE + Fore.YELLOW + "3" + Fore.RESET, "Synopsis"],
                [Fore.BLUE + Fore.YELLOW + "4" + Fore.RESET, "Exit"]
            ]
            for option in menu_options:
                print(option[0] + ": " + option[1])

            choice = input(f"{Fore.BLUE}Enter your choice: " + Fore.RESET)

            if choice == "1":
                while True:
                    new_language = input(Fore.BLUE + Style.BRIGHT + "Enter the language (es, en or br): " + Fore.RESET)
                    if new_language in ["es", "en", "br"]:
                        language = new_language
                        break
                    else:
                        print(Fore.RED + "Invalid language. Please try again." + Fore.RESET)
            elif choice == "2":
                manga_name = input(Fore.BLUE + Style.BRIGHT + "Enter the name of the manga: " + Fore.RESET)
                if manga_name:
                    url = f"https://{language}.ninemanga.com/search/?wd={manga_name}"
                    response = requests.get(url)
                    bs = BeautifulSoup(response.content, "html.parser")
                    manga_links = bs.find_all("a", {"class": "bookname"})
                    manga_list = [(i+1, manga.text, manga.get("href")) for i, manga in enumerate(manga_links)]
                    if manga_list:
                        print(Fore.BLUE + "Manga List:" + Fore.RESET)
                        for manga in manga_list:
                            print(f"{manga[0]}.{manga[1]}")
                        return manga_list
                    else:
                        print(Fore.RED + "No results found. Please try again." + Fore.RESET)
                else:
                    print(Fore.RED + "The manga name cannot be empty. Please try again." + Fore.RESET)
            elif choice == "3":
                self.sinopsis(language)
            elif choice == "4":
                print(Fore.RED + "Exiting the program..." + Fore.RESET)
                sys.exit()
            else:
                print(Fore.RED + "Invalid choice. Please try again." + Fore.RESET)

    def sinopsis(self, language):
        try:
            manga_name = input(Fore.BLUE + Style.BRIGHT + "Ingresa el nombre del manga: " + Fore.RESET)
            if manga_name:
                url = f"https://{language}.ninemanga.com/search/?wd={manga_name}"
                response = requests.get(url)
                bs = BeautifulSoup(response.content, "html.parser")
                manga_links = bs.find_all("a", {"class": "bookname"})
                manga_list = [(i+1, manga.text, manga.get("href")) for i, manga in enumerate(manga_links)]
                if manga_list:
                    for manga in manga_list:
                        print(f"{manga[0]}. {manga[1]}")
                    manga_number = input(Fore.BLUE + "Ingresa el número del manga del que deseas obtener la sinopsis: " + Fore.RESET)
                    if manga_number.isdigit():
                        manga_number = int(manga_number)
                        if manga_number >= 1 and manga_number <= len(manga_list):
                            manga_url = manga_list[manga_number - 1][2]
                            response = requests.get(manga_url)
                            bs = BeautifulSoup(response.content, "html.parser")
                            sinopsis = bs.find("p", itemprop="description")
                            estatus = bs.find("a", {"class" : "red"})
                            genres = bs.find("li", itemprop="genre").find_all("a")
                            genre_list = [genre.text for genre in genres]
                            if sinopsis:
                                sinopsis_text = sinopsis.text.strip().replace("\n", "")
                                status_text = estatus.text.strip()
                                genre_text = ", ".join(genre_list)
                                print(Fore.GREEN + "Sinopsis:\n" + Fore.RESET + sinopsis_text)
                                print(Fore.GREEN + "\nStatus:\n" + Fore.RESET + status_text)
                                print(Fore.GREEN + "\nGéneros:\n" + Fore.RESET + genre_text)
                            else:
                                print(Fore.RED + "Sinopsis no encontrada para este manga." + Fore.RESET)
                        else:
                            print(Fore.RED + "Número de manga inválido. Por favor, inténtalo de nuevo." + Fore.RESET)
                    else:
                        print(Fore.RED + "Entrada inválida. Por favor, ingresa un número." + Fore.RESET)
                else:
                    print(Fore.RED + "No se encontraron resultados. Por favor, inténtalo de nuevo." + Fore.RESET)
            else:
                print(Fore.RED + "El nombre del manga no puede estar vacío. Por favor, inténtalo de nuevo." + Fore.RESET)
        except Exception as e:
            print(Fore.RED + "Ha ocurrido un error durante el proceso: " + str(e) + Fore.RESET)


    def get_chapters(self, manga_list):
      while True:
        choice = input(Fore.BLUE + "Select a manga from the list: " + Fore.RESET)
        if choice.isdigit():
            choice = int(choice)
            if 1 <= choice <= len(manga_list):
                selected_manga = manga_list[choice-1][2]
                response = requests.get(selected_manga)
                bs = BeautifulSoup(response.content, "html.parser")
                chapters = bs.find_all("a", {"class": "chapter_list_a"})

                if not chapters:
                    selected_manga += "?waring=1"
                    response = requests.get(selected_manga)
                    bs = BeautifulSoup(response.content, "html.parser")
                    chapters = bs.find_all("a", {"class": "chapter_list_a"})

                # Invertir la lista de capítulos
                chapters.reverse()

                chapter_list = [(i+1, chapter.text, chapter.get("href")) for i, chapter in enumerate(chapters)]

                for chapter in chapter_list:
                    print(f"{chapter[0]}: {chapter[1]}")

                return chapter_list
            else:
                print(Fore.RED + "Invalid choice. Please try again." + Fore.RESET)
        elif choice.lower() == "back":
            return None
        else:
            print(Fore.RED + "Invalid choice. Please try again." + Fore.RESET)

    def pictures_from_chapters(self, content: str, response: requests.Response, chapter_title):
     title= re.sub(r'[\\/*?:"<>|]', "", chapter_title)
     print("\n")
     print(Fore.GREEN + f'{title}: \n' + Fore.RESET)
     bs = BeautifulSoup(content, "html.parser")
     container = bs.find("select", {"id": "page"})
     options = container.find_all("option")
     num = options[-1]  # Obtener el último número de página
     count = num.text
     count = count.split('/')[-1]
     total = math.ceil(len(options) / 10)
     pages = total
     images_url = []

     def get_page_content(page):
         url = f'{response.url[:-5]}-10-{page}.html'
         while True:
             try:
                 content, _ = self.get_url(url)
                 return content
             except requests.exceptions.RequestException:
                 print(Fore.RED + "The connection was lost. Waiting ..." + Fore.RESET)
                 time.sleep(5) # Esperar 5 segundos antes de intentar nuevamente

     with concurrent.futures.ThreadPoolExecutor() as executor:
         page_contents = executor.map(get_page_content, range(1, total + 1))

     for content in page_contents:
         bs = BeautifulSoup(content, "html.parser")
         images_url += [img.get("src") for img in bs.find_all("img", {"class": "manga_pic"})]
         #print(Fore.YELLOW + f"Obtained image links...{len(images_url)}", end="\r" + Fore.RESET)
     return images_url
    def get_url(self, url):
     headers = {
         'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'
     }

     with requests.get(url, headers=headers) as response:
         if response.status_code == 200:
             return response.content, response
         else:
             return None, None

    def download_images(self, images_url, chapter_title):
        folder_name = re.sub(r'[\\/*?:"<>|]', "", chapter_title)
        Path(folder_name).mkdir(parents=True, exist_ok=True)

        def download_image(url, index, bar):
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'
            }
            try:
                response = requests.get(url, headers=headers)
                response.raise_for_status()  # Raise an exception if there is an error in the response
                with open(f"{folder_name}/{index+1}.jpg", "wb") as f:
                    f.write(response.content)
                bar.update(1)
            except requests.exceptions.RequestException as e:
                print(Fore.RED + "\rThe connection was lost. Waiting..." + Fore.RESET, end="\r")
                time.sleep(5)
                download_image(url, index, bar)

       # print("\n")
        total_images = len(images_url)
        with tqdm(total=total_images, desc="Downloading...", bar_format='{l_bar}{bar} | {n_fmt}/{total_fmt} ', ncols=53) as bar:
            num_files = len(list(Path(folder_name).glob('*.jpg')))
            i = num_files
            bar.update(i)

            with ThreadPoolExecutor(max_workers=10) as executor:
                futures = []
                while i < total_images:
                    url = images_url[i]
                    if Path(folder_name, f'{i+1}.jpg').exists():
                        bar.update(1)
                        i += 1
                        continue
                    try:
                        future = executor.submit(download_image, url, i, bar)
                        futures.append(future)
                        i += 1
                    except requests.exceptions.RequestException as e:
                        print(Fore.RED + "The connection was lost. Waiting..." + Fore.RESET, end="\r")
                        time.sleep(5)
                        print("\r" + " " * 80 + "\r", end="\r")  # Clear the error message and show the progress bar

                for future in futures:
                    while not future.done():
                        time.sleep(1)
                        if not future.running():
                            future.result()

        self.create_cbz(folder_name, chapter_title)
        shutil.rmtree(folder_name)

        print(Fore.GREEN + "\nDownload completed!" + Fore.RESET)
       # self.search_manga()

    def create_cbz(self, folder_name, chapter_title):
       name = re.sub(r'[\\/*?:"<>|]', "", chapter_title)
       cbz_file = f"{name}.cbz"
       with zipfile.ZipFile(cbz_file, "w") as zf:
         for foldername, subfolders, filenames in os.walk(folder_name):
             for filename in filenames:
                 filepath = os.path.join(foldername, filename)
                 zf.write(filepath, arcname=filename)
a = TuClase()
manga_list = a.search_manga()
chapter_list = a.get_chapters(manga_list)
while True:
    choice = input(Fore.BLUE + "Select a chapter from the list: " + Fore.RESET)
    if choice == "back":
        # Retroceder a la lista de mangas
        manga_list = a.search_manga()
        chapter_list = a.get_chapters(manga_list)
        continue
    if choice.isdigit():
        choice = int(choice)
        if 1 <= choice <= len(chapter_list):
            chapter_title = chapter_list[choice-1][1]
            chapter_url = chapter_list[choice-1][2]
            content, response = a.get_url(chapter_url)
            images_url = a.pictures_from_chapters(content, response,chapter_title)
            a.download_images(images_url, chapter_title )
            break
        else:
            print(Fore.RED + "Invalid choice. Please try again." + Fore.RESET)
    elif ',' in choice:
        chapters = choice.split(',')
        if len(chapters) == 2 and chapters[0].isdigit() and chapters[1].isdigit():
            start = int(chapters[0])
            end = int(chapters[1])
            if 1 <= start <= end <= len(chapter_list):
                for i in range(start, end+1):
                    chapter_title = chapter_list[i-1][1]
                    chapter_url = chapter_list[i-1][2]
                    content, response = a.get_url(chapter_url)
                    images_url = a.pictures_from_chapters(content, response,chapter_title)
                    a.download_images(images_url, chapter_title )
                break
            else:
              print(Fore.RED + "Invalid choice. Please try again." + Fore.RESET)
        else:
          print(Fore.RED + "Invalid choice format. Please try again." + Fore.RESET)
    else:
      print(Fore.RED + "Choice cannot be empty. Please try again." + Fore.RESET)

In [ ]:
import shutil
import os

# Directorio donde están los archivos CBZ
source_dir = '/content/'

# Directorio de destino en Google Drive
dest_dir = '/content/drive/MyDrive/CBZFILES'

# Crea el directorio de destino si no existe
os.makedirs(dest_dir, exist_ok=True)

# Mueve todos los archivos CBZ
for file_name in os.listdir(source_dir):
    if file_name.endswith('.cbz'):
        shutil.move(os.path.join(source_dir, file_name), dest_dir)
